In [1]:
import numpy as np
from rotated_surface_code import RotatedSurfaceCode
from decoder import BPDecoder, RelayBPDecoder
from ldpc import BpDecoder as ldpc_BPDecoder
from ldpc import BpOsdDecoder

In [2]:
code = RotatedSurfaceCode(d=5)
error_rate = np.array([0.1] * code.n)
ex, ez, sx, sz = code.sample_error_and_syndrome_with_code_capacity_model(N=10, p=error_rate, seed=42)

# BP decoder (implementation from `ldpc` package) for decoding X errors

In [5]:
decoder = ldpc_BPDecoder(
    pcm=code.Hz,
    error_rate=0.1,
    max_iter=code.n,
    bp_method="minimum_sum",
)

for i in range(10):
    e, s = ex[i], sz[i]
    print("error:     ", e)
    print("syndrome:  ", s)
    ehat = decoder.decode(s)
    print("decoded:   ", ehat)
    print("converged? ", np.all(code.Hz @ ehat % 2 == s))

error:      [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
syndrome:   [1 0 0 0 0 0 0 0 0 0 0 0]
decoded:    [1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
converged?  False
error:      [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
syndrome:   [0 0 0 1 0 0 0 1 0 0 0 0]
decoded:    [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
converged?  False
error:      [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
syndrome:   [0 1 0 1 0 0 0 1 0 0 1 0]
decoded:    [0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
converged?  True
error:      [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
syndrome:   [0 1 0 1 0 0 0 1 0 0 0 1]
decoded:    [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
converged?  True
error:      [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
syndrome:   [0 1 0 0 0 0 0 0 1 0 0 0]
decoded:    [1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
converged?  False
error:      [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
syndrome:   [0 1 0

# BP+OSD decoder (implementation from `ldpc` package)

In [11]:
decoder = BpOsdDecoder(
    pcm=code.Hz,
    error_rate = 0.1,
    bp_method = 'minimum_sum',
    max_iter = code.n,
    # schedule = 'serial',
    osd_method = 'osd_cs',
    osd_order = 2
)

for i in range(10):
    e, s = ex[i], sz[i]
    print("error:     ", e)
    print("syndrome:  ", s)
    ehat = decoder.decode(s)
    print("decoded:   ", ehat)
    print("converged? ", np.all(code.Hz @ ehat % 2 == s))

error:      [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
syndrome:   [1 0 0 0 0 0 0 0 0 0 0 0]
decoded:    [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
converged?  True
error:      [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
syndrome:   [0 0 0 1 0 0 0 1 0 0 0 0]
decoded:    [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
converged?  True
error:      [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
syndrome:   [0 1 0 1 0 0 0 1 0 0 1 0]
decoded:    [0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
converged?  True
error:      [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
syndrome:   [0 1 0 1 0 0 0 1 0 0 0 1]
decoded:    [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
converged?  True
error:      [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
syndrome:   [0 1 0 0 0 0 0 0 1 0 0 0]
decoded:    [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
converged?  True
error:      [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
syndrome:   [0 1 0 0 

# BP decoder

In [ ]:
prior = np.array([0.1] * code.n)

bp = BPDecoder(
    H=code.Hz,
    prior=prior
)

for i in range(10):
    print("sample ", i)
    e, s = ex[i], sz[i]
    print("syndrome:      ", s)
    print("true error:    ", e)
    ehat, marginal = bp.decode(s)
    print("decoded error: ", ehat)
    print("")


sample  0
syndrome:       [1 0 0 0 0 0 0 0 0 0 0 0]
true error:     [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded error:  None

sample  1
syndrome:       [0 0 0 1 0 0 0 1 0 0 0 0]
true error:     [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
decoded error:  None

sample  2
syndrome:       [0 1 0 1 0 0 0 1 0 0 1 0]
true error:     [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
decoded error:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

sample  3
syndrome:       [0 1 0 1 0 0 0 1 0 0 0 1]
true error:     [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
decoded error:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]

sample  4
syndrome:       [0 1 0 0 0 0 0 0 1 0 0 0]
true error:     [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded error:  None

sample  5
syndrome:       [0 1 0 0 0 0 1 1 0 1 0 1]
true error:     [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
decoded error:  None

sample  6
syndrome:       [0 0 0 0 0 1 0 1 0 0 0 0]
tr

# RelayBP Decoder

In [8]:
prior = np.array([0.1] * code.n)
num_sol = 1
max_leg = 301

mem_strength = -0.25 + 0.85 / 2 * np.random.uniform(-1, 1, size=(max_leg, code.n))
mem_strength[0, :] = 0.35

max_iter_list = [80] + [60] * (max_leg - 1)

relay_bp = RelayBPDecoder(
    H=code.Hz,
    prior=prior,
    num_sol=num_sol,
    max_leg=max_leg,
    mem_strength=mem_strength,
    max_iter_list=max_iter_list
)

for i in range(10):
    print("sample ", i)
    e, s = ex[i], sz[i]
    print("syndrome:      ", s)
    print("true error:    ", e)
    ehat = relay_bp.decode(s)
    print("decoded error: ", ehat)
    print("")


sample  0
syndrome:       [1 0 0 0 0 0 0 0 0 0 0 0]
true error:     [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded error:  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

sample  1
syndrome:       [0 0 0 1 0 0 0 1 0 0 0 0]
true error:     [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
decoded error:  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]

sample  2
syndrome:       [0 1 0 1 0 0 0 1 0 0 1 0]
true error:     [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
decoded error:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

sample  3
syndrome:       [0 1 0 1 0 0 0 1 0 0 0 1]
true error:     [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
decoded error:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]

sample  4
syndrome:       [0 1 0 0 0 0 0 0 1 0 0 0]
true error:     [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded error:  [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

sample  5
syndrome:       [0 1 0 0 0 0 1 1 0 1 0 1]
tru